In [32]:
!git clone https://github.com/sahal-mulki/geo-llm/

fatal: destination path 'geo-llm' already exists and is not an empty directory.


In [33]:
import pandas as pd

In [38]:
def prompt_generator(addresses):
    prompt = (
        "Guess where this image was taken, get creative."
        "Here's some AI-generated addresses that could be wrong to get you started. Choose the best out of these if you're uncertain about the image:\n"
        f"1. {addresses[0]}\n"
        f"2. {addresses[1]}\n"
        f"3. {addresses[2]}\n"
        f"4. {addresses[3]}\n"
        f"5. {addresses[4]}"
    )
    
    return prompt

def image_info_finder(image_name, dataset):
    
    if "yfcc" in dataset:
        df = pd.read_csv("/kaggle/working/geo-llm/geoclip_predictions_yfcc4k.csv")
        addresses = [df[df["Image"] == image_name]["Prediction 1 Address"].values[0], df[df["Image"] == image_name]["Prediction 2 Address"].values[0], df[df["Image"] == image_name]["Prediction 1 Address"].values[0], df[df["Image"] == image_name]["Prediction 3 Address"].values[0], df[df["Image"] == image_name]["Prediction 4 Address"].values[0], df[df["Image"] == image_name]["Prediction 5 Address"].values[0]]
    elif "im2gps" in dataset:
        df = pd.read_csv("/kaggle/working/geo-llm/geoclip_predictions_im2gps3k.csv")
        addresses = [df[df["Image"] == image_name]["Prediction 1 Address"].values[0], df[df["Image"] == image_name]["Prediction 2 Address"].values[0], df[df["Image"] == image_name]["Prediction 1 Address"].values[0], df[df["Image"] == image_name]["Prediction 3 Address"].values[0], df[df["Image"] == image_name]["Prediction 4 Address"].values[0], df[df["Image"] == image_name]["Prediction 5 Address"].values[0]]
        
    return addresses



In [39]:
print(prompt_generator(image_info_finder("1000269685_e60e9cdfb4_1125_78841376@N00.jpg", "im2gps")))

You are an Image OSINT Investigator. I will provide you with an image and it's your job to determine where it was taken. By the end of your investigation you have to have at least a rough guess of the image's location. DECIDE ON ONE FINAL LOCATION, EVEN IF IT'S WRONG, JUST GUESS. Here's some AI-generated addresses that could be wrong to get you started. Choose one of these if you're uncertain:
1. Santa Cruz de Tenerife, Canary Islands, Spain
2. Santa Cruz de Tenerife, Canary Islands, Spain
3. Santa Cruz de Tenerife, Canary Islands, Spain
4. Santa Cruz de Tenerife, Canary Islands, Spain
5. Mexico


In [40]:
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [ ]:
df = pd.read_csv('/kaggle/input/llms-for-image-geolocation-benchmarks/archive(2)/im2gps3k_places365/im2gps3k_places365.csv')

from tqdm import tqdm

jsonl_reqs_for_im2gps = []
# Iterate over the rows of the DataFrame
for index, row in tqdm(df.iterrows(), total=1000):
    
    prompt = (
        "You are an Image OSINT Investigator. I will provide you with an image and it's your job to determine where it was taken. "
        "By the end of your investigation you have to have at least a rough guess of the image's location. "
        "DECIDE ON ONE FINAL LOCATION, EVEN IF IT'S WRONG, JUST GUESS. "
             )
    
    try:
        boilerplate = {
          "custom_id": "im2gps3k_" + row["IMG_ID"],
          "method": "POST",
          "url": "/v1/chat/completions",
          "body": {
            "model": "gpt-4o",
            "messages": [
              {
                "role": "system",
                "content": prompt
              },
              {
                "role": "user",
                "content": prompt_generator(image_info_finder(row["IMG_ID"], "im2gps"))
              }
            ],
            "max_tokens": 1000
          }
        }

        jsonl_reqs_for_im2gps.append(boilerplate)
    except TypeError:
        pass

 21%|██        | 207/1000 [00:09<00:35, 22.43it/s]

In [ ]:
len(jsonl_reqs_for_im2gps)